**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [1]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops

2023-01-26 13:36:42.083105: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-26 13:36:42.252642: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-26 13:36:42.252665: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-26 13:36:43.001282: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [3]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

In [4]:
# Преобразуйте метки классов в one_hot формат
y_train = tf.one_hot(y_train, CLASSES)
y_val = tf.one_hot(y_val, CLASSES)

2023-01-26 13:36:44.781956: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-26 13:36:44.782234: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-26 13:36:44.782285: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-01-26 13:36:44.782326: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-01-26 13:36:44.785247: W tensorflow/c

In [5]:
y_train = np.reshape(y_train.numpy(), (50000, 100))
y_val = np.reshape(y_val.numpy(), (10000, 100))

In [6]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [7]:
import keras
from tensorflow.keras import layers
from keras.models import Sequential

In [8]:
from tensorflow.keras.layers import Input, Activation, Conv2D, BatchNormalization, Flatten, Dense, MaxPooling2D, Dropout, SeparableConv2D

In [54]:
# Задайте архитектуру модели
model = Sequential([
    Input(shape=[32,32,3]),
    
    Conv2D(32, 3, padding='same', strides = (2, 2)),
    BatchNormalization(),
    Activation('leaky_relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(.05),
    
    SeparableConv2D(64, 5, padding='same', strides = (2, 2)),
    BatchNormalization(),
    Activation('leaky_relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(.05),
    
    Flatten(),
    Dense(185),
    BatchNormalization(),
    Activation('selu'),
    #Dropout(.1),
    Dense(CLASSES),
    BatchNormalization(),
    Activation('softmax')
])

In [55]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

2023-01-26 13:59:58.220001: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-26 13:59:58.220245: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-01-26 13:59:58.220294: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-01-26 13:59:58.220539: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-26 13:59:58.220750: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gp

le:
node name | # float_ops
_TFProfRoot (--/705.73k flops)
  sequential_10/conv2d_10/Conv2D (442.37k/442.37k flops)
  sequential_10/dense_20/MatMul (94.72k/94.72k flops)
  sequential_10/separable_conv2d_10/separable_conv2d (65.54k/91.14k flops)
    sequential_10/separable_conv2d_10/separable_conv2d/depthwise (25.60k/25.60k flops)
  sequential_10/dense_21/MatMul (37.00k/37.00k flops)
  sequential_10/batch_normalization_40/FusedBatchNormV3 (16.58k/16.58k flops)
  sequential_10/conv2d_10/BiasAdd (8.19k/8.19k flops)
  sequential_10/max_pooling2d_20/MaxPool (8.19k/8.19k flops)
  sequential_10/batch_normalization_41/FusedBatchNormV3 (2.43k/2.43k flops)
  sequential_10/max_pooling2d_21/MaxPool (1.02k/1.02k flops)
  sequential_10/separable_conv2d_10/BiasAdd (1.02k/1.02k flops)
  sequential_10/activation_43/Softmax (500/500 flops)
  sequential_10/batch_normalization_42/batchnorm/Rsqrt (370/370 flops)
  sequential_10/batch_normalization_43/batchnorm/Rsqrt (200/200 flops)
  sequential_10/batch_no

In [56]:
print(f"FLOPs: {(flops / 1e6):.4f}e6")

FLOPs: 0.7057e6


In [57]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 16, 16, 32)        896       
                                                                 
 batch_normalization_40 (Bat  (None, 16, 16, 32)       128       
 chNormalization)                                                
                                                                 
 activation_40 (Activation)  (None, 16, 16, 32)        0         
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 8, 8, 32)         0         
 g2D)                                                            
                                                                 
 dropout_13 (Dropout)        (None, 8, 8, 32)          0         
                                                                 
 separable_conv2d_10 (Separa  (None, 4, 4, 64)       

In [58]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.experimental.AdamW(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [59]:
# обучение модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=50
)

Epoch 1/50
391/391 [==============================] - 6s 13ms/step - loss: 3.5054 - accuracy: 0.1830 - val_loss: 3.8008 - val_accuracy: 0.1534
Epoch 2/50
391/391 [==============================] - 5s 13ms/step - loss: 2.9600 - accuracy: 0.2742 - val_loss: 3.2183 - val_accuracy: 0.2244
Epoch 3/50
391/391 [==============================] - 5s 13ms/step - loss: 2.7278 - accuracy: 0.3156 - val_loss: 2.7173 - val_accuracy: 0.3210
Epoch 4/50
391/391 [==============================] - 5s 13ms/step - loss: 2.5783 - accuracy: 0.3464 - val_loss: 2.5791 - val_accuracy: 0.3541
Epoch 5/50
391/391 [==============================] - 5s 14ms/step - loss: 2.4692 - accuracy: 0.3665 - val_loss: 2.5319 - val_accuracy: 0.3634
Epoch 6/50
391/391 [==============================] - 5s 13ms/step - loss: 2.3847 - accuracy: 0.3848 - val_loss: 2.5170 - val_accuracy: 0.3600
Epoch 7/50
391/391 [==============================] - 5s 13ms/step - loss: 2.3218 - accuracy: 0.3983 - val_loss: 2.4108 - val_accuracy: 0.3900